In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
def download_file(url, path, download_path):
    """Download the dataset"""
    if not os.path.isdir(path):
        os.makedirs(path)
    urllib.request.urlretrieve(url, download_path)
    if (os.path.isfile(download_path)):
        print(f'Download path: {download_path}')

In [3]:
import os
import zipfile
import pandas as pd
import numpy as np
from six.moves import urllib

DOWNLOAD_ROOT = "http://www.sportstoto.com.my/"
LOTTO_PATH = os.path.join(os.getcwd(), "test_data", "lotto")

_4D_URL = DOWNLOAD_ROOT + "upload/4D.zip"
_4D_ZIP = os.path.join(LOTTO_PATH, "4D.zip")
download_file(_4D_URL, LOTTO_PATH, _4D_ZIP)

Download path: /home/alpha2phi/workspace/alpha2phi/jupyter-notebooks/nbs/test_data/lotto/4D.zip


In [4]:
downloaded_file = _4D_ZIP
df = pd.read_csv(downloaded_file, header=0, sep=',', quotechar='"', dtype=str)
df['DrawDate']= pd.to_datetime(df['DrawDate'], format='%Y%m%d')
df.head(2)

,DrawNo,DrawDate,1stPrizeNo,2ndPrizeNo,3rdPrizeNo,SpecialNo1,SpecialNo2,SpecialNo3,SpecialNo4,SpecialNo5,...,ConsolationNo1,ConsolationNo2,ConsolationNo3,ConsolationNo4,ConsolationNo5,ConsolationNo6,ConsolationNo7,ConsolationNo8,ConsolationNo9,ConsolationNo10
0,040792,1992-05-06,0019,1124,0592,0950,2479,7139,3114,4609,...,5311,1949,1606,3775,6226,1271,7455,7227,9258,0407
1,040892,1992-05-07,0905,3591,8690,7202,6796,4959,2008,7506,...,4562,0823,8938,1933,5378,9909,7586,9765,4640,9963


In [5]:
def apply_checksum(row):
    """Calculate unique checksum for each date"""
    cksum = calc_checksum(row["1stPrizeNo"])
    cksum = cksum + calc_checksum(row["2ndPrizeNo"])
    cksum = cksum + calc_checksum(row["3rdPrizeNo"])
    for idx in range(10):
        col = "SpecialNo" + str(idx + 1)
        cksum = cksum + calc_checksum(row[col])
    for idx in range(10):
        col = "ConsolationNo" + str(idx + 1)
        cksum = cksum + calc_checksum(row[col])
    return cksum

def calc_checksum(no):
    return ( (int(no[0]) * 10) + (int(no[1]) * 100) + (int(no[2]) * 1000) + (int(no[3]) * 10000) ) 
    
    
df["CheckSum"] = df.apply(apply_checksum, axis=1)
df.tail(100)


,DrawNo,DrawDate,1stPrizeNo,2ndPrizeNo,3rdPrizeNo,SpecialNo1,SpecialNo2,SpecialNo3,SpecialNo4,SpecialNo5,...,ConsolationNo2,ConsolationNo3,ConsolationNo4,ConsolationNo5,ConsolationNo6,ConsolationNo7,ConsolationNo8,ConsolationNo9,ConsolationNo10,CheckSum
4788,523621,2021-02-06,6314,0187,4359,8481,8312,7936,3007,2590,...,8745,0670,5710,1288,1090,4130,7198,8742,2092,947000
4789,523721,2021-02-07,2478,7044,7391,7785,7415,1170,9699,3812,...,1402,7268,1538,6629,1913,2894,2255,5792,9029,1184220
4790,523821,2021-02-09,4725,0100,6483,1302,1162,2825,1965,0717,...,6224,6235,7281,0226,9752,0045,3942,8230,7329,966760
4791,523921,2021-02-10,4959,9923,0350,2784,4053,0482,4883,1636,...,7524,5522,3169,4350,8415,1427,1805,3086,2079,1191870
4792,524021,2021-02-13,2502,3079,7311,9708,1024,5968,0636,3488,...,9830,8647,9425,8651,8035,4687,0305,8881,5055,1142010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4883,538321,2021-12-04,2607,3836,8540,4848,8080,8018,6807,6238,...,6982,3010,8217,5414,7137,0145,2421,2362,4997,1164210
4884,538421,2021-12-05,1635,7492,6019,5295,2741,0482,7006,8132,...,7333,9239,7467,7016,5916,2106,8584,1789,4330,1154650
4885,538521,2021-12-07,4639,6922,6676,5271,1713,0473,0227,2671,...,0955,9792,6117,6947,9976,5670,1412,7253,9914,1054250
4886,538621,2021-12-08,8128,1211,0287,7649,4181,5522,9241,8835,...,9287,6095,5490,9125,6038,9326,1350,5679,4781,1084260


## Time Series Forecast

In [ ]:
!pip install -Uq pystan==2.19.1.1
!pip install -Uq prophet